<a href="https://colab.research.google.com/github/dheemanthAthreya/AI-Lab/blob/main/1BM22CS347_Week8_FOL_ForwardReasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Given facts and rules
known_facts = {
    "American(Robert)": True,
    "Missile(T1)": True,
    "Owns(A, T1)": True,
    "Enemy(A, America)": True,
}

inference_rules = [
    # Rule 1: Missiles are weapons
    {"if": ["Missile(x)"], "then": ["Weapon(x)"]},

    # Rule 2: Selling weapons to an enemy country makes someone a criminal
    {"if": ["American(p)", "Weapon(q)", "Sells(p, q, r)", "Hostile(r)"], "then": ["Criminal(p)"]},

    # Rule 3: All missiles were sold to country A by Robert
    {"if": ["Missile(x)", "Owns(A, x)"], "then": ["Sells(Robert, x, A)"]},

    # Rule 4: The enemy of America is considered hostile
    {"if": ["Enemy(x, America)"], "then": ["Hostile(x)"]},
]

# Query to verify
target_query = "Hostile(A)"

# Function to match a condition with a fact
def match_condition(condition, fact):
    cond_pred, cond_args = condition.split("(")
    fact_pred, fact_args = fact.split("(")

    if cond_pred != fact_pred:
        return None  # Predicates must match

    cond_args = cond_args[:-1].split(",")
    fact_args = fact_args[:-1].split(",")

    if len(cond_args) != len(fact_args):
        return None

    mappings = {}

    for var, val in zip(cond_args, fact_args):
        if var.islower():  # Variable
            mappings[var] = val
        elif var != val:  # Constant mismatch
            return None

    return mappings

# Function to substitute variables in a statement
def substitute_in_statement(statement, mappings):
    pred, args = statement.split("(")
    args = args[:-1].split(",")
    updated_args = [mappings.get(arg, arg) for arg in args]
    return f"{pred}({', '.join(updated_args)})"

# Forward chaining mechanism
def forward_chaining_inference(known_facts, inference_rules, target_query):
    inferred_facts = set(known_facts.keys())  # Initialize with known facts

    while True:
        new_infer = set()

        for rule in inference_rules:
            # Check if all "if" conditions are satisfied
            conditions = rule["if"]
            candidate_subs = [{}]

            for condition in conditions:
                temp_subs = []

                for fact in inferred_facts:
                    substitutions = match_condition(condition, fact)
                    if substitutions is not None:
                        for existing_sub in candidate_subs:
                            combined_sub = existing_sub.copy()
                            combined_sub.update(substitutions)
                            temp_subs.append(combined_sub)

                if not temp_subs:
                    break
                candidate_subs = temp_subs

            # If all conditions are met, infer the "then" conclusions
            if candidate_subs:
                for subs in candidate_subs:
                    for conclusion in rule["then"]:
                        inferred_fact = substitute_in_statement(conclusion, subs)
                        new_infer.add(inferred_fact)

        # If target query is inferred, return True
        print(new_infer)
        if target_query in new_infer:
            return True

        # If no new facts can be inferred, return False
        if not new_infer - inferred_facts:
            return False

        inferred_facts.update(new_infer)

# Run the forward chaining process and check the result
if forward_chaining_inference(known_facts, inference_rules, target_query):
    print(f"The query '{target_query}' is proven.")
else:
    print(f"The query '{target_query}' cannot be proven.")


{'Sells(Robert,  T1,  A)', 'Weapon(T1)', 'Criminal(Robert)', 'Hostile(A)'}
The query 'Hostile(A)' is proven.
